# PHYS 1600 HW1
Tim Zhao, B01307256, Feb 10, 2021

In [2]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

## #1 
The Debye model of a solid is used to estimate the contribution of atomic vibrations (phonons) to the specific heat capacity. In the Debye model, the specific heat capacity of a solid is given by the following integral expression: 
$C_v=9k_B\bigg(\frac{T}{\theta_D}\bigg)\int_{0}^{\theta_D/T} \frac{x^4e^x}{(e^x-1)^2}dx$ 
where kB is Boltzmann’s constant and θD is the Debye temperature, a property of solids that depends on their density and atomic bonding strength.

a) Write a Python function to calculate $C_V$ for a given material and temperature. Your program should take the density, sample volume, Debye Temperature, temperature upper limit, and required accuracy as inputs, and output the heat capacity between $T = 0$ and the upper temperature limit. Use an adaptive Simpson’s rule to evaluate the integral to the required accuracy.

In [3]:
def adapt_simp(f, start, stop, accuracy, args = None):
    """
    f - function to integrate
    start - lower limit
    stop - upper limit
    accuracy - desired error range
    """
    
    #chose initial N and error
    h = 0.1
    error = 100
    
    
    #find S0 w/ simpsons
    x_list = np.arange(start,stop+h,h)
    S0 = (1/3)*(f(start,*args)+f(stop,*args)+2*f(x_list[2:-2:2].sum()))
    T0 = (2/3)* f(x_list[1:-1:2].sum())
    F0 = h*(S0+2*T0)
    
    #make empty lists to store each rounds of calculation
    S_list=[S0,]
    T_list=[T0,]
    F_list=[F0,]

    while error > accuracy:
        
        #half step size
        h /= 2
        x_list = np.arange(start,stop+h,h)
        
        #adaptive simpson
        T = (2/3)* f(x_list[1:-1:2],*args).sum()
        S = S_list[-1]+T_list[-1]
        F = h*(S+2*T)
        
        S_list.append(S)
        T_list.append(T)   
        F_list.append(F)
        
        #check error
        error = abs((1/15)*(F_list[-1]-F_list[-2]))
    
    return F_list[-1]

In [4]:
def polynomial(x,*args):
    
    """
    A general polynomial the order of which is determined by the size
    of the array of coefficients
    Arguments:
    x -- vector of x values
    args -- an array of coeficients, the size of this array determines the order
    of the polynomial
    example: polynomial(x,2,3) will generate the line: f(x) = 2 + 3x
    or 
    polynomial(x,1,0,0.1) will generate: f(x) = 1 +0.1*x^2, fixed conflicts
    """
    f = 0*x
    for (i,a) in enumerate(args):
        f += a*x**(i)
    
    return f

In [2]:
print(adapt_simp(polynomial,0,100,0.0001,args=[0,1]))

NameError: name 'adapt_simp' is not defined

In [ ]:
def Debye(rho,V,theta_D,T_max,accu):
    """
    adaptive simpson's rule
    
    rho - density
    V - volume
    theta_D - Debye Temperature
    T_max - Temperature upper limit
    accu - required accuracy
    return array of Temperature and Heat Capacity between T=0 and T_max
    """
    
    #start with small N
    N=1000
    
    #check accuracy
    while error>accu:
        
        #adaptive simpson
    

In [3]:
#General ODE solver with Runge-Kutta 4
def RK4(f,t0,t_f,dt,x0):
    """
    f - a function, takes 2 variables (t,x), RHS of ODE
    t0 - lower limit of independent variable
    t_f - upper limit of independent variable
    dt - step size
    x0 - initial value 
    return - array of t and x
    """
    
    # number of time steps, n must be an integer
    n  = int(np.ceil((t_f-t0)/dt))
    
    # create empty lists to store values
    x_list=np.zeros(n, np.longdouble)
    t_list=np.zeros(Q_list.size)
    
    # set initial conditions at time zero
    x_list[0] = x0
    t_list[0] = t0
    
    # RK4
    for ii in range(n-1):
            k1 = dt*f(x_list[ii],t_list[ii])
            k2 = dt*f(x_list[ii]+0.5*k1,t_list[ii]+dt/2)
            k3 = dt*f(x_list[ii]+0.5*k2,t_list[ii]+dt/2)
            k4 = dt*f(x_list[ii]+0.5*k3,t_list[ii]+dt/2)
            
            # update value of Q at each time step and step time by dt
            x_list[ii+1]=x_list[ii]+(k1+2*k2+2*k3+k4)/6
            t_list[ii+1]=t_list[ii]+dt
            
    return t_list, Q_list